In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_kraken_orderbooks_from_file
from hydradx.model.processing import get_omnipool_data_from_file
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb
import hydradx.model.amm.arbitrage_agent_new as arb2

from pprint import pprint

import json


In [2]:
with open('config/arbconfig.txt', 'r') as json_file:
    arb_list = json.load(json_file)
    
for d in arb_list:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

ob_objs = {}
order_book_asset_list = []
buffer = {}

get_live_markets = True

if get_live_markets:
    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[arb_cfg['tkn_ids']] = arb_cfg['buffer']
        if tkn_pair not in ob_objs:
            order_book_url = 'https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1]
            ob_objs[tkn_pair] = get_kraken_orderbook(tkn_pair, order_book_url)
            for tkn in tkn_pair:
                if tkn not in order_book_asset_list:
                    order_book_asset_list.append(tkn)
                    
else:
    ob_objs = get_kraken_orderbooks_from_file("data/input/")
    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[arb_cfg['tkn_ids']] = arb_cfg['buffer']
        for tkn in tkn_pair:
            if tkn not in order_book_asset_list:
                order_book_asset_list.append(tkn)    
                


In [3]:
RPC="wss://hydradx-rpc.dwellir.com"
N = 24
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, N, archive=True)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
buffer2 = {(asset_map[k[0]], asset_map[k[1]]): v for k, v in buffer.items()}
pprint(tokens)
pprint(buffer2)


{'ASTR': {'LRNA': 9426.083139248882, 'liquidity': 4487123.1018498875},
 'BNC': {'LRNA': 6603.946708737461, 'liquidity': 729547.862157833},
 'CFG': {'LRNA': 9231.929536764723, 'liquidity': 600988.2603155252},
 'DAI': {'LRNA': 12896.518469037715, 'liquidity': 338250.81980178953},
 'DAI001': {'LRNA': 31189.317908758978, 'liquidity': 816351.8852662736},
 'DOT': {'LRNA': 73055.18732006775, 'liquidity': 413807.9998085864},
 'GLMR': {'LRNA': 5351.198130481999, 'liquidity': 664616.1758693877},
 'HDX': {'LRNA': 20197.434429212783, 'liquidity': 113691495.84240358},
 'INTR': {'LRNA': 8316.189716962293, 'liquidity': 12845383.67597828},
 'USDT': {'LRNA': 84987.58325140616, 'liquidity': 2231172.007803},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41584.505065373174, 'liquidity': 31.25332448},
 'WETH': {'LRNA': 21426.620304704305, 'liquidity': 309.21785254735175},
 'WETH001': {'LRNA': 70997.18376287296, 'liquidity': 1027.4506953779776},
 'ZTG': {'LRNA': 4579.8402

In [4]:


cex_fee = 0.0016
# buffer = 0.0010

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0
# buffer = 0.0


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [5]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    if base_id in asset_map and quote_id in asset_map:
        tkn_pair = (asset_map[base_id], asset_map[quote_id])
        order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'DAI'): ('ASTR', 'USD'),
 ('ASTR', 'DAI001'): ('ASTR', 'USD'),
 ('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'DAI'): ('BNC', 'USD'),
 ('BNC', 'DAI001'): ('BNC', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'DAI'): ('CFG', 'USD'),
 ('CFG', 'DAI001'): ('CFG', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'DAI'): ('DOT', 'USD'),
 ('DOT', 'DAI001'): ('DOT', 'USD'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC001'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('DOT', 'WETH001'): ('DOT', 'ETH'),
 ('DOT', 'iBTC'): ('DOT', 'XBT'),
 ('GLMR', 'DAI'): ('GLMR', 'USD'),
 ('GLMR', 'DAI001'): ('GLMR', 'USD'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'DAI'): ('HDX', 'USD'),
 ('HDX', 'DAI001'): ('HDX', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('WBTC001', 'DAI'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI001'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'USDT'): ('XXBT', 'ZUSD'),
 ('WETH', 'DAI'): ('XETH', 'ZUSD'),
 ('WETH', 'DAI001'): ('XETH', 'ZUSD'),
 ('WETH', 'USDT'): ('XETH', 'ZUS

In [6]:
cex1 = cex.copy()
op_state1 = op_state.copy()
all_swaps1 = arb.get_arb_swaps(op_state1, cex1, order_book_map, buffer=buffer2)
# tot_swaps = 0
# for tkn_pair in all_swaps1:
#     if all_swaps1[tkn_pair]:
#         print(tkn_pair)
#         print(all_swaps1[tkn_pair])
#         tot_swaps += len(all_swaps1[tkn_pair])
# print(tot_swaps)
print(all_swaps1)
print(len(all_swaps1))

cex2 = cex.copy()
op_state2 = op_state.copy()
all_swaps2 = arb2.get_arb_swaps(op_state2, cex2, order_book_map, buffer=buffer2)
print("strategy 2")
pprint(all_swaps2)
pprint(len(all_swaps2))

[{'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'WETH', 'price': 1804.05, 'amount': 0.3355811462402344, 'min_buy': 607.5094705725313}, 'cex': {'trade': 'buy', 'buy_asset': 'XETH', 'sell_asset': 'ZUSD', 'price': 1804.9520249999998, 'amount': 0.3355811462402344}}, {'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'iBTC', 'price': 34880.7, 'amount': 0.008, 'min_buy': 888.5106203647853}, 'cex': {'trade': 'buy', 'buy_asset': 'XXBT', 'sell_asset': 'ZUSD', 'price': 34898.140349999994, 'amount': 0.008}}, {'dex': {'trade': 'sell', 'buy_asset': 'DAI', 'sell_asset': 'iBTC', 'price': 34899.0, 'amount': 0.013279693603515628, 'min_buy': 465.2984113362432}, 'cex': {'trade': 'buy', 'buy_asset': 'XXBT', 'sell_asset': 'ZUSD', 'price': 34916.449499999995, 'amount': 0.013279693603515628}}, {'dex': {'trade': 'sell', 'buy_asset': 'DAI001', 'sell_asset': 'iBTC', 'price': 34899.0, 'amount': 0.004284836149017793, 'min_buy': 149.90211766046286}, 'cex': {'trade': 'buy', 'buy_asset': 'XXB

In [7]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex1 = cex.copy()
cex2 = cex.copy()

arb.execute_arb(state1, cex1, agent1, all_swaps1)
arb.execute_arb(state2, cex2, agent2, all_swaps2)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb.calculate_profit(init_agent, agent2, asset_map)

asset_map_omnipool = {
    'ETH': 'WETH',
    'XETH': 'WETH',
    'XXBT': 'WBTC',
    'BTC': 'WBTC',
    'ZUSD': 'USDT',
    'USD': 'USDT',
    'USDC': 'USDT',
    'DAI': 'USDT',
    'USDT001': 'USDT',
    'DAI001': 'USDT',
    'WETH001': 'WETH',
    'WBTC001': 'WBTC',
    'iBTC': 'WBTC',
    'XBT': 'WBTC',
}


profit1_omnipool = arb.calculate_profit(init_agent, agent1, asset_map_omnipool)
profit2_omnipool = arb.calculate_profit(init_agent, agent2, asset_map_omnipool)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Omnipool spot")
print(sum([profit1_omnipool[asset] * state1.price(state1, asset, 'USDT') for asset in profit1_omnipool]))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Omnipool spot")
print(sum([profit2_omnipool[asset] * state2.price(state2, asset, 'USDT') for asset in profit2_omnipool]))
            

strategy 1 profit
{'HDX': 0, 'USD': 4.6321938037872314, 'BTC': 0.0, 'ETH': 0.0, 'DOT': 0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0, 'INTR': 0}
strategy 1 profit at Omnipool spot
4.6321938037872314
strategy 2 profit
{'HDX': 0, 'USD': 6.8264347314834595, 'BTC': 6.079673767089844e-06, 'ETH': 0.0, 'DOT': 0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0, 'INTR': 0}
strategy 2 profit at Omnipool spot
7.0107580107699725
